In [2]:
!pip install PyPDF2
import PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00


In [3]:
import csv
import re
import pandas as pd
import statsmodels.api as sm

## IFM RCP 4.5 - Landes

La base de données étant en format pdf, le code suivant permet de les retranscrire dans un fichier csv

Tout marche bien mais on a pas dans ces bases les IFM de 2006 à 2022, qui nous sont utiles pour pouvoir construire le modèle probabiliste.

In [4]:
# Ouvrir le fichier PDF
with open('IFM RCP 4.5, Landes.pdf', 'rb') as f:
    # Créer un objet PdfReader
    pdf_reader = PyPDF2.PdfReader(f)
    
    # Lire chaque page du PDF et extraire le texte
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

# Afficher le texte extrait
#print(text)


In [5]:
# Définir une expression régulière pour extraire les données de chaque ligne
pattern = re.compile(r'(\d+);([\d.-]+);([\d.-]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);([^;]+);')

# Ouvrir le fichier PDF extrait
with open('text.txt', 'w') as file:
    file.write(text)

# Ouvrir le fichier texte extrait
with open('text.txt', 'r') as file:
    # Créer un fichier CSV pour écrire les données
    with open('ifm45_landes.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=[
            'Point', 'Latitude', 'Longitude', 'Contexte', 'Période', 'Saison',
            'NORNSV23', 'NORNSV24', 'NORNSV25', 'NORIFM20', 'NORIFM40', 'NORIFM60',
            'NORIS400', 'NORIS700', 'NORIS1000'
        ])
        
        writer.writeheader()
        
        # Lire chaque ligne du fichier extrait
        for line in file:
            # Vérifier si la ligne correspond au pattern
            match = pattern.match(line)
            if match:
                # Extraire les données de la ligne
                data = {
                    'Point': match.group(1),
                    'Latitude': match.group(2),
                    'Longitude': match.group(3),
                    'Contexte': match.group(4),
                    'Période': match.group(5),
                    'Saison': match.group(6),
                    'NORNSV23': match.group(7),
                    'NORNSV24': match.group(8),
                    'NORNSV25': match.group(9),
                    'NORIFM20': match.group(10),
                    'NORIFM40': match.group(11),
                    'NORIFM60': match.group(12),
                    'NORIS400': match.group(13),
                    'NORIS700': match.group(14),
                    'NORIS1000': match.group(15)
                }
                
                # Écrire les données dans le fichier CSV
                writer.writerow(data)


In [6]:
# Lire le fichier CSV avec pandas
df_ifm45 = pd.read_csv('ifm45_landes.csv')

# Afficher les premières lignes du DataFrame pour vérifier les données
print(df_ifm45.head())


   Point  Latitude  Longitude Contexte Période  Saison  NORNSV23  NORNSV24  \
0   4034   43.5074    -1.3690   RCP4.5      H1       2         0         0   
1   4034   43.5074    -1.3690   RCP4.5      H1       3         9         1   
2   4034   43.5074    -1.3690   RCP4.5      H1       4         5         0   
3   4035   43.5107    -1.2704   RCP4.5      H1       2         0         0   
4   4035   43.5107    -1.2704   RCP4.5      H1       3        16         2   

  NORNSV25  NORIFM20  NORIFM40  NORIFM60  NORIS400  NORIS700  NORIS1000  
0     -999         1         0         0         0         0       -999  
1        0         7         0         0        23         1          0  
2     -999         2         0         0        18         1       -999  
3     -999         1         0         0         0         0       -999  
4        0         8         0         0        22         1          0  


On va donc faire de même pour le fichier ifm donnees 2022 rcp4.5.txt qui à priori contient les IFM recherchés

In [7]:
### UPLOAD LE FICHIER AVANT ###

# Initialiser une liste vide pour stocker les données
data = []

# Ouvrir le fichier texte pour la lecture
with open('ifm donnees 2022 rcp4.5.txt', 'r') as file:
    for line in file:
        # Ignorer les lignes de commentaire
        if not line.startswith('#'):
            # Diviser la ligne pour obtenir les différentes valeurs
            columns = line.strip().split(';')
            # Ajouter les valeurs à la liste de données
            data.append(columns)

# Créer un DataFrame à partir de la liste de données
df_ifm45 = pd.DataFrame(data, columns=['Date', 'Latitude', 'Longitude', 'ifm', 'ifm-max'])

# Convertir les colonnes de données en types appropriés
df_ifm45['Date'] = pd.to_datetime(df_ifm45['Date'], format='%Y%m%d')
df_ifm45['Latitude'] = pd.to_numeric(df_ifm45['Latitude'])
df_ifm45['Longitude'] = pd.to_numeric(df_ifm45['Longitude'])
df_ifm45['ifm'] = pd.to_numeric(df_ifm45['ifm'])
df_ifm45['ifm-max'] = pd.to_numeric(df_ifm45['ifm-max'])

# Afficher les premières lignes du DataFrame pour vérification
print(df_ifm45.head())


        Date  Latitude  Longitude  ifm  ifm-max
0 2006-01-01   43.5074   -1.36829  0.7      0.8
1 2006-01-02   43.5074   -1.36829  0.6      0.7
2 2006-01-03   43.5074   -1.36829  0.6      0.7
3 2006-01-04   43.5074   -1.36829  0.9      0.9
4 2006-01-05   43.5074   -1.36829  0.9      1.0


## Lien entre les IFM et la survenance des feux de forêt

In [8]:
data = 'combine.csv'

df = pd.read_csv('combine.csv', sep=';')
print(df.head())

# Filtrer les lignes où la valeur de la colonne "Département" est égale à 40
bdiff_landes = df[df['Département'] == '40']

# Afficher les premières lignes du DataFrame filtré
print(bdiff_landes.head())

   Année  Numéro Département Code INSEE       Nom de la commune  \
0   2006       2          01      01148                  Dortan   
1   2006       3          01      01321                Revonnas   
2   2006       4          01      01032               Béligneux   
3   2006       5          01      01109               Collonges   
4   2006       6          01      01384  Saint-Rambert-en-Bugey   

  Date de première alerte Origine de l'alerte  \
0        27/02/2006 13:56         Indéterminé   
1        15/03/2006 16:02         Indéterminé   
2        16/03/2006 17:03         Indéterminé   
3        27/03/2006 14:30         Indéterminé   
4        24/04/2006 16:43         Indéterminé   

   Moyens de première intervention  Surface parcourue (m2)  \
0                              NaN                   10000   
1                              NaN                   10000   
2                              NaN                   10000   
3                              NaN                   1

**Régression linéaire du nombre de feux sur les IFM**

*Préparation des données*

In [9]:
# Créer une copie de bdiff_landes 
bdiff_landes = bdiff_landes.copy()

# Convertir la colonne 'Date de première alerte' en datetime
bdiff_landes['Date de première alerte'] = pd.to_datetime(bdiff_landes['Date de première alerte'], format='%d/%m/%Y %H:%M')

# Extraire la date sans l'heure pour compter le nombre de feux par jour
bdiff_landes['Date'] = bdiff_landes['Date de première alerte'].dt.date

# Compter le nombre de feux par jour
feux_par_jour = bdiff_landes.groupby('Date').size().reset_index(name='Nombre de feux')

*Mettre les données en relation*

In [10]:
# Convertir la colonne 'Date' dans df_ifm45 en format date sans heure pour la fusion
df_ifm45['Date'] = pd.to_datetime(df_ifm45['Date']).dt.date

# Fusionner les données sur la colonne 'Date'
donnees_analyse = pd.merge(df_ifm45, feux_par_jour, on='Date', how='inner')


*Régression*

In [11]:
# Préparer les variables pour la régression
X = donnees_analyse[['ifm']]  # Variable explicative
y = donnees_analyse['Nombre de feux']  # Variable à expliquer

# Ajouter une constante à la variable explicative pour le terme d'interception
X = sm.add_constant(X)

# Effectuer la régression linéaire
modele = sm.OLS(y, X).fit()

# Afficher le résumé du modèle
print(modele.summary())


                            OLS Regression Results                            
Dep. Variable:         Nombre de feux   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3936.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        16:37:15   Log-Likelihood:            -2.6502e+05
No. Observations:              150800   AIC:                         5.301e+05
Df Residuals:                  150798   BIC:                         5.301e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4299      0.005    306.291      0.0

**On rajoute la surface totale brûlée voir si c'est expliqué par l'IFM**

In [12]:
# Calculer le nombre de feux et la surface brûlée totale par jour
feux_par_jour = bdiff_landes.groupby('Date').agg({'Nom de la commune': 'size', 'Surface parcourue (m2)': 'sum'}).reset_index().rename(columns={'Nom de la commune': 'Nombre de feux', 'Surface parcourue (m2)': 'Surface brûlée totale'})

donnees_analyse = pd.merge(df_ifm45, feux_par_jour, on='Date', how='inner')

# Préparer les variables pour la régression
X = donnees_analyse[['ifm', 'Nombre de feux']]  # Variables explicatives
y = donnees_analyse['Surface brûlée totale']  # Variable à expliquer

# Ajouter une constante à la variable explicative pour le terme d'interception
X = sm.add_constant(X)

# Effectuer la régression linéaire
modele = sm.OLS(y, X).fit()

# Afficher le résumé du modèle
print(modele.summary())


                              OLS Regression Results                             
Dep. Variable:     Surface brûlée totale   R-squared:                       0.001
Model:                               OLS   Adj. R-squared:                  0.001
Method:                    Least Squares   F-statistic:                     47.29
Date:                   Thu, 28 Mar 2024   Prob (F-statistic):           2.94e-21
Time:                           16:38:47   Log-Likelihood:            -2.0976e+06
No. Observations:                 150800   AIC:                         4.195e+06
Df Residuals:                     150797   BIC:                         4.195e+06
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const         